In [11]:
#!pip install sentence-transformers lightgbm torchvision optuna tqdm --quiet



import pandas as pd

import numpy as np

import gc

import re, os, requests, torch, json

from tqdm import tqdm

from PIL import Image

from io import BytesIO

from sentence_transformers import SentenceTransformer

from torchvision import models, transforms

from sklearn.model_selection import train_test_split

import lightgbm as lgb

import optuna

In [12]:
import pandas as pd
KAGGLE_INPUT_PATH = '/kaggle/input/sample-dataset'
KAGGLE_WORKING_PATH = '/kaggle/working/'

try:
    train_df = pd.read_csv(os.path.join(KAGGLE_INPUT_PATH, 'Sample_train.csv'))
    test_df  = pd.read_csv(os.path.join(KAGGLE_INPUT_PATH, 'sample_test.csv'))
    print("Train shape:", train_df.shape)
    print("Test shape:", test_df.shape)
except FileNotFoundError:
    raise FileNotFoundError(f"Please ensure Sample_train.csv and Sample_test.csv are in the dataset at: {KAGGLE_INPUT_PATH}")

Train shape: (1000, 4)
Test shape: (100, 3)


In [13]:
def clean_text(text):
    if pd.isnull(text):
        return ""
    text = re.sub(r'[^a-zA-Z0-9 ]', ' ', text)
    return text.lower().strip()

def extract_ipq(text):
    if pd.isnull(text): return 1
    text = text.lower()
    patterns = [
        r'pack of (\d+)', r'(\d+)\s*count', r'set of (\d+)',
        r'(\d+)\s*pack', r'(\d+)\s*(pcs|pieces|units?)'
    ]
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return int(match.group(1))
    return 1

def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(numerator / (denominator + 1e-8)) * 100

def lgb_smape(y_true, y_pred):
    return 'smape', smape(y_true, y_pred), False  # lower = better

for df in [train_df, test_df]:
    df['clean_text'] = df['catalog_content'].apply(clean_text)
    df['ipq'] = df['catalog_content'].apply(extract_ipq)

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

model_text = SentenceTransformer('all-MiniLM-L6-v2')

resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1]).eval().to(device)
img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def extract_image_features(url):
    try:
        response = requests.get(url, timeout=10)
        img = Image.open(BytesIO(response.content)).convert('RGB')
        img_t = img_transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            features = resnet(img_t).cpu().numpy().flatten()
        return features
    except Exception:
        return np.zeros(2048)

def get_or_create_embeddings(df, text_model, image_extractor, prefix):
    # Files will be saved to the persistent /kaggle/working/ directory
    text_file = os.path.join(KAGGLE_WORKING_PATH, f"{prefix}_text_embeds.npy")
    img_file = os.path.join(KAGGLE_WORKING_PATH, f"{prefix}_img_embeds.npy")

    if os.path.exists(text_file):
        print(f"Loading {text_file}")
        text_embeds = np.load(text_file)
    else:
        print(f"Encoding text for {prefix} ...")
        text_embeds = text_model.encode(df['clean_text'].tolist(), show_progress_bar=True)
        np.save(text_file, text_embeds)

    if os.path.exists(img_file):
        print(f"Loading {img_file}")
        img_embeds = np.load(img_file)
    else:
        print(f"Extracting images for {prefix} ...")
        img_feats = [image_extractor(url) for url in tqdm(df['image_link'], desc=f"{prefix} images")]
        img_embeds = np.vstack(img_feats)
        np.save(img_file, img_embeds)
        # Clean up to save memory
        del img_feats
        gc.collect()

    return text_embeds, img_embeds

train_text_embeds, train_img_embeds = get_or_create_embeddings(train_df, model_text, extract_image_features, "train")
test_text_embeds,  test_img_embeds  = get_or_create_embeddings(test_df,  model_text, extract_image_features, "test")

Using device: cpu
Loading /kaggle/working/train_text_embeds.npy
Extracting images for train ...




train images:   0%|          | 0/1000 [00:00<?, ?it/s]

train images:   0%|          | 1/1000 [00:00<04:15,  3.91it/s]

train images:   0%|          | 2/1000 [00:00<04:26,  3.74it/s]

train images:   0%|          | 3/1000 [00:00<04:10,  3.98it/s]

train images:   0%|          | 4/1000 [00:00<03:59,  4.15it/s]

train images:   0%|          | 5/1000 [00:01<04:17,  3.86it/s]

train images:   1%|          | 6/1000 [00:01<04:48,  3.44it/s]

train images:   1%|          | 7/1000 [00:01<04:44,  3.49it/s]

train images:   1%|          | 8/1000 [00:02<04:55,  3.36it/s]

train images:   1%|          | 9/1000 [00:02<04:31,  3.65it/s]

train images:   1%|          | 10/1000 [00:02<04:17,  3.85it/s]

train images:   1%|          | 11/1000 [00:02<04:07,  3.99it/s]

train images:   1%|          | 12/1000 [00:03<04:14,  3.88it/s]

train images:   1%|▏         | 13/1000 [00:03<04:05,  4.02it/s]

train images:   1%|▏         | 14/1000 [00:03<04:00,  4.10it/s]

train images:   2%|▏         | 15/1000 [0

Encoding text for test ...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Extracting images for test ...


test images: 100%|██████████| 100/100 [00:31<00:00,  3.17it/s]


In [16]:
train_ipq = train_df['ipq'].values.reshape(-1, 1)
test_ipq  = test_df['ipq'].values.reshape(-1, 1)

X_train_all = np.hstack([train_text_embeds, train_img_embeds, train_ipq])
X_test_all  = np.hstack([test_text_embeds,  test_img_embeds,  test_ipq])
y = train_df['price'].values

print("Final Feature Shape:", X_train_all.shape)


Final Feature Shape: (1000, 2433)


In [17]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train_all, y, test_size=0.15, random_state=42
)

In [18]:
optuna_dir = os.path.join(KAGGLE_WORKING_PATH, 'optuna_studies')
os.makedirs(optuna_dir, exist_ok=True)
optuna_db_path = os.path.join(optuna_dir, 'product_price_tuning.db')

storage_name = f"sqlite:///{optuna_db_path}"
study_name = "lgbm_price_prediction_mae"

try:
    study = optuna.load_study(study_name=study_name, storage=storage_name)
    print(" Loaded existing Optuna study.")
except KeyError:
    study = optuna.create_study(direction='minimize', study_name=study_name, storage=storage_name)
    print(" New Optuna study created.")

[I 2025-10-14 13:10:17,058] A new study created in RDB with name: lgbm_price_prediction_mae


 New Optuna study created.


In [29]:
def objective(trial):
    params = {
        'objective': 'mae',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 31, 128),
        'max_depth': trial.suggest_int('max_depth', 5, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 200),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'n_estimators': 3000,
        'random_state': 42,
        'n_jobs': -1
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric=lgb_smape,
        callbacks=[lgb.early_stopping(100, verbose=False)]
    )
    y_pred = model.predict(X_val)
    y_pred[y_pred < 0] = 0
    print(smape(y_val, y_pred))
    return smape(y_val, y_pred)
    


In [30]:
study.optimize(objective, n_trials=25, show_progress_bar=True)

best_params_file = os.path.join(optuna_dir, 'best_params.json')
with open(best_params_file, 'w') as f:
    json.dump(study.best_trial.params, f, indent=4)
print(f" Best parameters saved to {best_params_file}")


  0%|          | 0/25 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.131592 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 620196
[LightGBM] [Info] Number of data points in the train set: 850, number of used features: 2433
[LightGBM] [Info] Start training from score 14.950000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga


                                                             
train images:  15%|█▍        | 146/1000 [26:08<04:34,  3.11it/s]

67.06504700017538
[I 2025-10-14 13:27:17,538] Trial 25 finished with value: 67.06504700017538 and parameters: {'learning_rate': 0.03332212166187645, 'num_leaves': 92, 'max_depth': 12, 'min_child_samples': 77, 'subsample': 0.9617294729199353, 'colsample_bytree': 0.6042685123995635}. Best is trial 10 with value: 65.65902808568586.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.100826 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 620196
[LightGBM] [Info] Number of data points in the train set: 850, number of used features: 2433
[LightGBM] [Info] Start training from score 14.950000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 


                                                             
train images:  15%|█▍        | 146/1000 [26:12<04:34,  3.11it/s]

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
67.10404684489944
[I 2025-10-14 13:29:50,826] Trial 47 finished with value: 67.10404684489944 and parameters: {'learning_rate': 0.03551598658472757, 'num_leaves': 77, 'max_depth': 11, 'min_child_samples': 75, 'subsample': 0.9036707666102208, 'colsample_bytree': 0.7135624355634274}. Best is trial 10 with value: 65.65902808568586.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.101764 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 620196
[LightGBM] [Info] Number of data points in the train set: 850, number of used features: 2433
[LightGBM] [Info] Start training from score 14.950000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 


                                                             
train images:  15%|█▍        | 146/1000 [28:47<04:34,  3.11it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
66.32829300812976
[I 2025-10-14 13:29:56,699] Trial 48 finished with value: 66.32829300812976 and parameters: {'learning_rate': 0.04093964154916281, 'num_leaves': 70, 'max_depth': 12, 'min_child_samples': 167, 'subsample': 0.8603583218242022, 'colsample_bytree': 0.6190381693228592}. Best is trial 10 with value: 65.65902808568586.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.101366 seconds.
You can set `force_col_wise=true` to re


                                                             
train images:  15%|█▍        | 146/1000 [28:53<04:34,  3.11it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
65.76955665396966
[I 2025-10-14 13:30:02,602] Trial 49 finished with value: 65.76955665396966 and parameters: {'learning_rate': 0.04539720653789773, 'num_leaves': 83, 'max_depth': 8, 'min_child_samples': 88, 'subsample': 0.9274434499607571, 'colsample_bytree': 0.8815935448687511}. Best is trial 10 with value: 65.65902808568586.
 Best parameters saved to /kaggle/working/optuna_studies/best_params.json


In [22]:
with open(best_params_file, 'r') as f:
    best_params = json.load(f)

best_params.update({
    'objective': 'mae',
    'boosting_type': 'gbdt',
    'random_state': 42,
    'n_jobs': -1
})

final_model = lgb.LGBMRegressor(**best_params)
# Train on the entire dataset for the final model
final_model.fit(X_train_all, y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.120222 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 620201
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 2433
[LightGBM] [Info] Start training from score 14.950000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

LGBMRegressor(colsample_bytree=0.9353511452626933,
              learning_rate=0.04970071620795562, max_depth=12,
              min_child_samples=189, n_jobs=-1, num_leaves=85, objective='mae',
              random_state=42, subsample=0.9821713874808182)

In [24]:
test_pred = final_model.predict(X_test_all)
test_pred[test_pred < 0] = 0

submission = pd.DataFrame({
    'sample_id': test_df['sample_id'],
    'price': test_pred
})
# Submission file is saved to /kaggle/working/ which is the required output directory
submission.to_csv(os.path.join(KAGGLE_WORKING_PATH, 'submission.csv'), index=False)
print("\n submission.csv generated successfully in /kaggle/working/!")


 submission.csv generated successfully in /kaggle/working/!


In [27]:
from optuna.visualization import plot_optimization_history, plot_param_importances
plot_optimization_history(study).show()
plot_param_importances(study).show()

NameError: name 'smpe' is not defined